In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lmfit import Model
from scipy import stats
import time

import cekComputerLabs as cek

In [ ]:
# 1 Intro
# 2 Bomb
# 3 CV
# 4 Langmuir
# 5 MD
iLab = [5]

In [ ]:
# Intro lab
if 1 in iLab:
    numberOfRuns = 5
    files = ["lab1."+str(i)+".csv" for i in range(0,numberOfRuns)]

    for f in files:
        cek.introLaboratory2(7,10,f)
        time.sleep(1)
        print("-------")
        
    fig,ax=plt.subplots()
    for f in files:
        df = pd.read_csv(f)
        x = df.iloc[:,0]
        y = df.iloc[:,1]
        ax.plot(x,y,label=f)
    plt.legend()
    plt.show()

In [ ]:
# Bomb calorimetry
if 2 in iLab:
    numberOfRuns = 5
    files = ["lab2."+str(i)+".csv" for i in range(0,numberOfRuns)]

    mass = []
    for f in files:
        mass.append(cek.bombCalorimetryLab("Benzoic acid",f))
        time.sleep(1)
        print("-------")
        
    fig,ax=plt.subplots()
    for f in files:
        df = pd.read_csv(f)
        x = df.iloc[:,0]
        y = df.iloc[:,1]
        ax.plot(x,y,label=f)
    plt.legend()
    plt.show()

In [ ]:
# Crystal violet
if 3 in iLab:
    OH = list(range(10,60,10))        
    files = ["lab3."+str(i)+".csv" for i in range(0,len(OH))]
    
    exp = []
    for i in range(0,len(OH)):
        exp.append(cek.crystalVioletLab(files[i],10,OH[i],50-OH[i],25))
        time.sleep(1)
        print(exp[-1])
        print("-------")

    fig,ax=plt.subplots()
    for f in files:
        df = pd.read_csv(f)
        x = df.iloc[:,0]
        y = df.iloc[:,1]
        ax.plot(x,y,label=f)
    plt.legend()
    plt.show()

In [ ]:
# Lagmuir
if 4 in iLab:
    T = list(range(25,66,10))        
    files = ["lab4."+str(i)+".csv" for i in range(0,len(T))]
    for i in range(0,len(T)):
        exp.append(cek.surfaceAdsorptionLab(files[i],T[i],0.1,50,1000,200))
        time.sleep(1)
        print(exp[-1])
        print("-------")

    fig,ax=plt.subplots()
    for f in files:
        df = pd.read_csv(f)
        x = df.iloc[:,3]
        y = df.iloc[:,2]/1000/584.910641/0.1 - x
        ax.plot(x,y,label=f)
    plt.legend()
    plt.show()

In [ ]:
if 5 in iLab:
#     exp = []
    files = []
    for T in range(283,373,10):
        f = "thermo."+str(T)+".csv"
        files.append(f)
#         exp.append(cek.waterMolecularDynamics(f,T=T,simTime=100))
        print("-------")

    avg = []
    avgError = []
    temp = []
    fig,ax=plt.subplots()
    for i,f in enumerate(files):
        df = pd.read_csv(f)
        df = df[ df["Time (ps)"]>4] 
        x = df["Time (ps)"]
        d = df["Density (g/mL)"]
        t = df["Temperature (K)"]
        v = df["Box Volume (nm^3)"]

        mean = np.mean(v)
        stdev = np.std(v,ddof=1)
        sterr = stdev / np.sqrt(len(x))
        tvalue = stats.t.ppf(q=1-.05/2,df=len(x)-1)
        CI = tvalue * sterr
        avg.append(mean)
        avgError.append(CI)

        temp.append(exp[i]["T"])
        ax.plot(x,v,label=f)
    plt.legend()
    plt.show()

    for i in range(0,len(files)):
        print("Average Volume (nm^3): {:.3f} +/- {:.3f}".format(avg[i],avgError[i]))
    print("-------")

    lnV = np.log(np.array(avg))

    def func(x,a,b):
        return a + b*x

    lmodel = Model(func)
    params = lmodel.make_params(a=1,b=1)
    result = lmodel.fit(lnV,params,x=temp,weights=1/np.array(avgError))
    print(result.fit_report())
    print("-------")

    tValues = np.linspace(273,373,50)
    yValues = result.eval(x=tValues)
    dely =  result.eval_uncertainty(x=tValues)*2

    fig,ax = plt.subplots()
    ax.scatter(temp,lnV,label="data")
    ax.plot(tValues,yValues,label="fit")
    ax.fill_between(tValues,yValues-dely,yValues+dely,label="95% CI",alpha=0.5,color='gray')
    plt.show()